 # Generating Tempogram from Novelty Onset Function                  
 ###        Prepared by: Rachit Gupta
 
In this Notebook novelty onset functions are used to generate tempogram representation. Tempogram is similar to a spectrogram but is a time-tempo representation of a given time-dependent signal.

For computing tempograms, we typically first extract a novelty curve. The peaks of this curve yield good indicators for note onsets. In a second step, local periodic patterns are derived from the novelty curve using fourier transform method

Steps to get a tempogram using Fourier transform method:
- Compute a spectrogram (STFT) of the novelty curve
- Convert frequency axis (given in Hertz) into
  tempo axis (given in BPM)
- Magnitude spectrogram indicates local tempo 

## Importing Modules:

In [ ]:
import os,sys
import matplotlib.pyplot as plt
import numpy as np
from essentia.standard import *
from scipy.signal import get_window
import IPython.display as ipd
import urllib.request
from IPython.display import Image
from scipy.signal import medfilt,argrelextrema
import IPython   


## Downloading Sound from Freesound:

In [ ]:
import sys,os
sys.path.append(os.path.join(os.path.dirname('__file__'), 'freesound-python/'))

import freesound
client = freesound.FreesoundClient()
#You are expected to place your token in the line below
client.set_token("...yourTokenGoesHere...")

maxNumFilePerInst=1

queryStrings=["127 Bpm Dub Techno Groove"]
for queryStr in queryStrings:
    results = client.text_search(query=queryStr,fields="id,name,previews,tags")
    cnt=0
    for sound in results:
        #print(sound.name)
        sound.retrieve_preview(".",sound.name+".wav")
        os.rename(sound.name+".wav",queryStr+"_"+str(cnt)+".wav")
        cnt=cnt+1
        if cnt>=maxNumFilePerInst:
            break

print("Files are copied! check your folder")

## Defining functions to calculate Onsets for Tempogram:
- Spectral Flux: Spectral flux is a measure of how quickly the power spectrum of a signal is changing, calculated by comparing the power spectrum for one frame against the power spectrum from the previous frame
- High Frequency Content: It is used to characterize the amount of high-frequency content in the signal. The magnitudes of the spectral bins are added together, but multiplying each magnitude by the bin "position" (proportional to the frequency). It accurately detects percussive events.

In [ ]:

def spectralFlux(x,windowSize,hopSize,winType="hann"):
    w = Windowing(type = winType)
    spectrum = Spectrum(size = windowSize)
    prevAmpSpec=np.zeros((1+int(windowSize/2),))
    sf=[]
    for frame in FrameGenerator(x, frameSize = windowSize, hopSize = hopSize):
        ampSpec = spectrum(w(frame))
        specDiff=ampSpec-prevAmpSpec
        h=(specDiff+np.abs(specDiff))/2#half-wave rectification
        
        sf.append(np.sum(h))
        
        prevAmpSpec = ampSpec
        
    SF=np.array(sf[1:])
    return SF

def highFreqCont(x,windowSize,hopSize,winType="hann"):
    w = Windowing(type = winType)
    spectrum = Spectrum(size = windowSize)

    hfc=[]
    for frame in FrameGenerator(x, frameSize = windowSize, hopSize = hopSize):
        ampSpec = spectrum(w(frame))
        hfc.append(np.dot(np.power(ampSpec,2),np.arange(ampSpec.size)))
    hfc=np.array(hfc) 
    # compute the derivative and half-wave rectification
    hfc_dev=hfc[1:]-hfc[:-1]#derivative
    hfc_dev=(hfc_dev+np.abs(hfc_dev))/2#half-wave rectification
    return hfc_dev





## Tempogram Representation:
Here, to present the basic idea we use two drawings from Meinhard Muller's music processing lecture notes
[Source for the Images](https://www.audiolabs-erlangen.de/content/05-fau/professor/00-mueller/02-teaching/2016w_mpa/2016_MuellerMeinard_LectureMusicProcessing_BeatTracking.pdf)

In [ ]:
Image(url="http://i63.tinypic.com/2e331n7.png")

In [ ]:

Image(url= "http://i67.tinypic.com/1zb89ea.png")

## Defining Fourier Tempogram function:

In [ ]:

def fourierTempo(onset,fwindSize,fhopSize):

        beatSpec = np.array([]).reshape(0,(fwindSize/2)+1)
        startIndexes=np.arange(0, onset.size-fwindSize,fhopSize, dtype=int)
        numWindows=startIndexes.size
    
        w = get_window(('hann'), fwindSize)
        spec=Spectrum()
    
        for k in range(numWindows):
            startInd=startIndexes[k]
            onset1=essentia.array(onset[startInd:startInd+fwindSize]*w)
            magn=spec(onset1)
            beatSpec = np.vstack((beatSpec,magn))

        

        return beatSpec


## Audio Files used:

In [ ]:
fs = 44100
windowSize=1024
hopSize=256
filename = '127 Bpm Dub Techno Groove_0.wav'
p = MonoLoader(filename=filename, sampleRate = fs)()
p = p/np.max(np.abs(p))
t1=np.arange(p.size)/float(fs)
#plotting signal 

#Getiing Specral Flux Onsets
Sf=spectralFlux(p,windowSize,hopSize);Sf=Sf/np.max(Sf)

#Getting HFC Onsets
hfc=highFreqCont(p,windowSize,hopSize);hfc=hfc/np.max(hfc)

print('Input Signal of File '+ filename)

#plotting 
f, axarr = plt.subplots(3,1,figsize=(20,10))
plt.subplots_adjust( bottom=.5, hspace=1,wspace=1,top=2)
    
#Plotting the Signal and Onsets using HFC and Spectral Flux method
axarr[0].plot(t1,p,label='Input Signal:')
axarr[1].plot(hfc,label=' Onsets High freq Content');axarr[1].axis('off');axarr[1].legend(loc=1)
axarr[2].plot(Sf,label=' Onsets Spectral Flux');axarr[2].axis('off');axarr[2].legend(loc=1)    
f.tight_layout()
plt.show()
IPython.display.Audio(p,rate=fs)

In [ ]:
filename = '../../../data/baris/varyingTempoBlues.mp3'
l = MonoLoader(filename = filename, sampleRate = fs)()
l = l/np.max(np.abs(l))
t2=np.arange(l.size)/float(fs)
#plotting signal 

#Getiing Specral Flux Onsets
Sf=spectralFlux(l,windowSize,hopSize);Sf=Sf/np.max(Sf)

#Getting HFC Onsets
hfc=highFreqCont(l,windowSize,hopSize);hfc=hfc/np.max(hfc)

print('Input Signal of File '+ filename)

#ploting onsets and tempogram
f, axarr = plt.subplots(3,1,figsize=(20,10))
plt.subplots_adjust( bottom=.5, hspace=1,wspace=1,top=2)
    
#Plotting onsets of Hfc
axarr[0].plot(t2,l,label='Input Signal:')
axarr[1].plot(hfc,label='Onsets High freq Content');axarr[1].axis('off');axarr[1].legend(loc=1)
axarr[2].plot(Sf,label=' Onsets Spectral Flux');axarr[2].axis('off');axarr[2].legend(loc=1)    
f.tight_layout()
plt.show()

IPython.display.Audio(l,rate=fs)


## Plotting Function for Tempogram:

In [ ]:


def plot(hfcfb,sffb,tempogram2,tempogram):
    
    f, axarr = plt.subplots(4,1,figsize=(10,20))
    plt.subplots_adjust( bottom=.5, hspace=1,wspace=1,top=2)
    
    

    #plotting  tempogram
    
    
    #Plotting Tempogram Hfc
    axarr[0].pcolormesh(np.transpose(hfcfb[:,:200]))
    axarr[0].set_title('Tempogram(Fourier) using HFC onsets of '+ file)
    axarr[0].set_ylabel('Tempo')
    axarr[0].set_xlabel('Time')
    

    #Plotting Tempogram using BpmHistogram 
    axarr[1].pcolormesh(np.transpose(tempogram2))
    axarr[1].set_title('Bpm Histogram Tempogram using HfC Onsets of '+ file)
    axarr[1].set_ylabel('Tempo(Bpm)')
    axarr[1].set_xlabel('Time')
    
    
    #Plotting Tempogram Spectral flux
    axarr[2].pcolormesh(np.transpose(sffb[:,:200]))
    axarr[2].set_title('Tempogram(Fourier) using Spectral Flux Onsets of '+ file)
    axarr[2].set_ylabel('Tempo')
    axarr[2].set_xlabel('Time')
    
    #Plotting Tempogram using BpmHistogram of Spectral Flux onsets
    axarr[3].pcolormesh(np.transpose(tempogram))
    axarr[3].set_title('BpmHistogram Tempogram using Spectral flux Onsets of'+file)
    axarr[3].set_ylabel('Tempo(Bpm)')
    axarr[3].set_xlabel('Time')

    f.tight_layout()
    plt.show()



## Calling  the fourier transform function and BpmHistogram from Essentia to generate Tempograms of Audio files:

In [ ]:
files=['127 Bpm Dub Techno Groove_0.wav','../../../data/baris/varyingTempoBlues.mp3']

fs = 44100
windowSize=1024
hopSize=256
fwindowsize=1024
fhopsize=100

for file in files:

    x = MonoLoader(filename = file, sampleRate = fs)()
    t=np.arange(x.size)/float(fs)
    
    x = x/np.max(np.abs(x))
    np.seterr(divide='ignore', invalid='ignore')
    tempo=BpmHistogram(maxBpm=560, minBpm = 30)
    
    #Getiing Specral Flux Onsets
    Sf=spectralFlux(x,windowSize,hopSize);Sf=Sf/np.max(Sf)

    #Getting HFC Onsets
    hfc=highFreqCont(x,windowSize,hopSize);hfc=hfc/np.max(hfc)
    
    #getting spectrum of HFC Onsets
    hfcfb=fourierTempo(hfc,fwindowsize,fhopsize)
    
    #gettin spectrum of Spectral Flux Onsets
    sffb=fourierTempo(Sf,fwindowsize,fhopsize)
       
    #Using Bpm Histogram from Essentia to get Tempogram 
    bpm, bpmCandidates, bpmMagnitudes, tempogram, frameBpms, ticks, ticksMagnitude, sinusoid= tempo(Sf)
    bpm2, bpmCandidates2, bpmMagnitudes2, tempogram2, frameBpms2, ticks2, ticksMagnitude2, sinusoid2= tempo(essentia.array(hfc))
    
    #calling plot function 
    plot(hfcfb,sffb,tempogram2,tempogram)
    
    

From the tempogram it can be seen that the tempo for the file "127 Bpm Dub Techno Groove_0.wav" is almost constant throughout the length of the audio track while it gradually increases for the file "varyingTempoBlues.mp3" for both the Hfc and Spectral Flux onsets.